In [ ]:
import time
import astropy
import itertools

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [ ]:
%matplotlib notebook

plt.style.use('dark_background')

# DESI y la supercomputadora más rápida de Occidente

Entender _cómo_ los 30 millones de galaxias estudiadas por DESI se formaron en el Universo es difícil, realmente difícil. De hecho, es tan difícil que DESI explota [Summit](https://www.olcf.ornl.gov/summit/), la supercomputadora más rápida del mundo en el Laboratorio Nacional de Oak Ridge para calcular cómo debería verse la distribución de las galaxias según el tipo de Energía Oscura:

<img src="images/summit.jpg" alt="Drawing" style="width: 800px;"/>

Con un costo de construcción de 325 millones de dólares, Summit es capaz de calcular $1.486 \times 10^{17}$ operaciones de suma y multiplicación por segundo, equivalente a $1.486 \times 10^{11}$ MegaFlops o MFLOPS. A modo de comparación, veamos qué proporciona Binder (necesitarás un poco de paciencia, tal vez quieras dejar esto para más adelante):

In [ ]:
_ = flops()

¡Así que Summit es al menos mil millones de veces más poderoso! Con Summit, podemos resolver los detalles más finos de la distribución de _materia oscura_ que trazan todas las galaxias:

<img src="images/abacus.png" alt="Drawing" style="width: 600px;"/>

Aquí, las regiones más brillantes significan las regiones más densas de materia oscura en el Universo, en las que esperamos encontrar más galaxias. El siguiente video muestra que hemos observado esta estructura predicha en la distribución de galaxias reales, observadas con experimentos antes de DESI:

In [ ]:
YouTubeVideo('08LBltePDZw', width=800, height=400)

[La Materia Oscura](https://es.wikipedia.org/wiki/Materia_oscura) es un elemento omnipresente en nuestro Universo, que constituye el 25% de la densidad (de energía) total. Con la Energía Oscura y los átomos comunes (materia bariónica) formando el resto. No sabemos casi nada sobre la materia oscura, más allá de su atracción gravitacional a otra materia y luz en el Universo.

Afortunadamente, las ecuaciones que describen la evolución de la materia oscura, a diferencia de la [formación compleja de galaxias](https://es.wikipedia.org/wiki/Formación_y_evolución_de_las_galaxias) (ver [también](https://www.space.com/15680-galaxies.html)), son relativamente simples para el Universo en el que vivimos. Todo lo que se requiere es rastrear la atracción gravitacional de las partículas de materia oscura (en un Universo en expansión).

Podemos predecir la evolución de la materia oscura muestreando la fuerza gravitacional, la velocidad y la posición con un conjunto de partículas (ficticias) que representan cada una un "grupo" de materia oscura con una masa total. Por supuesto, esto significa que no podemos resolver la distribución de la materia oscura dentro de estas regiones del tamaño de grupos, pero solo la distribución entre aglomerados conduce a la estructura que puede verse arriba. Con Summit, el grupo más pequeño que podemos resolver no está lejos de la masa del 'halo' de materia oscura que rodea el centro de nuestra propia [Vía Láctea](https://manuastronomo.wordpress.com/2019/03/11/hubble-y-gaia-pesan-con-precision-la-via-lactea/):

<img src="images/MilkyWay.jpg" alt="Drawing" style="width: 1000px;"/>

Para empezar, colocaremos inicialmente un conjunto de grupos en posiciones aleatorias dentro de un cubo 3D y les daremos velocidades iniciales cero. Las velocidades se generarán en momentos posteriores, ya que la atracción gravitacional ($1/r^2$) de una partícula hacia todas las demás provoca una aceleración neta.

In [ ]:
def init_dof(npt=1):
  #Crea un conjunto de partículas con posiciones aleatorias en una caja, 
  #las cuales predicen la distribución de materia oscura, como vemos arriba.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs, ys, zs)).T
  vel    = np.zeros_like(pos)              

  return  pos, vel

La fuerza gravitacional experimentada por cada partícula de materia oscura es $F = \frac{GmM}{r^2} \hat r$, la ley de gravitación Universal de [Newton](https://en.wikipedia.org/wiki/Isaac_Newton),  con la que puede que estés familiarizado. Solo necesitamos hacer un trabajo minucioso en la contabilidad requerida para calcular la fuerza total experimentada por una partícula debido a todas las demás:

In [ ]:
def g_at_particle(index, pos, mass, epsilon=1.0, doimages=True):
    # Ecuación 10 de http://www.skiesanduniverses.org/resources/KlypinNbody.pdf 
    # Para español puedes ver también https://rmf.smf.mx/pdf/rmf/38/5/38_5_701.pdf

    indices = np.arange(len(pos))
    rest    = pos[indices != index,:]

    #Aquí epsilón es un factor de control para prevenir que las partículas se acerquen, y la fuerza gravitacional sea infinita. 
    result  = mass * np.sum((rest - pos[index]) / ((rest - pos[index])**2. + epsilon**2.)**(3./2.), axis=0)

    if doimages:
        # Nuestra simulación asume condiciones de frontera periódicas, para la aceleración de cada partícula 
        # está la correspondiente aceleración de una imagen de la partícula producida al aplicar cambios periódicos en su posición. 
        shift  = np.array([-1, 0, 1]) 
        images = []

        for triple in itertools.product(shift, repeat=3):
            images.append(triple)
    
        images.remove((0, 0, 0))
    
        images = np.array(images)
    
        for image in images:
            result += mass * np.sum((rest + image - pos[index]) / ((rest + image - pos[index])**2. + epsilon**2.)**(3./2.), axis=0)

    return  result

En un experimento notable en 1941, Erik Holmberg utilizó el hecho de que el brillo de la luz decae con la distancia al mismo ritmo ($1/r^2$) que la gravedad. Para calcular la fuerza total sobre una 'partícula' en su 'simulación', Erik colocó una bombilla en la posición de cada partícula y calculó la fuerza efectiva sobre una partícula dada midiendo el brillo total en cada punto. La siguiente figura ilustra esta idea. ¡Intenta ejecutar la siguiente celda varias veces! Obtendrás un diseño aleatorio diferente de bombillas cada vez.

In [ ]:
fig, ax   = plt.subplots(1, 1, figsize=(5,5))

xmin, xmax, ymin, ymax = (0., 1., 0., 1.)

N         = 100
xx, yy    = np.meshgrid(np.linspace(xmin, xmax, N), np.linspace(ymin, ymax, N))
epsilon   = 0.1

weights   = np.zeros_like(xx) 

pos, vel  = init_dof(npt=10)

for par in pos:
  weights += 1. / ((xx - par[0])**2 + (yy - par[1])**2 + epsilon**2.)

ax.imshow(weights, extent=(xmin, xmax, ymin, ymax), cmap=plt.cm.afmhot, alpha=0.5, origin='lower')

ax.plot(xx[:,0], xx[:,1], '.', c='darkorange', alpha=0.75)
ax.tick_params(labelbottom=False, labelleft=False)  

ax.set_xlim(0., 1.)
ax.set_ylim(0., 1.)

plt.axis('off')

Este trabajo fue el concepto original de las simulaciones gravitacionales de 'n-cuerpos' que se describen aquí. ¡Es casi un crimen que solo 118 autores hayan hecho referencia a esta innovadora idea!

<img src="images/Holmberg.png" alt="Drawing" style="width: 800px;"/>

Hoy, dadas las mini supercomputadoras que tenemos a menudo al alcance de la mano, podemos determinar la distribución final de la materia oscura con mayor precisión con computadoras que con bombillas. Evolucionando una distribución homogénea inicial (una distribución casi uniforme de aglomeraciones de materia oscura, como el universo produjo en el Big Bang) bajo la gravedad, podemos predecir con precisión la ubicación de las galaxias (los lugares donde se forman los grupos de materia oscura más grandes).
 
Para hacer esto, solo necesitamos calcular la aceleración de cada partícula en una serie de pasos de tiempo y actualizar la velocidad y la posición de acuerdo con la aceleración que experimenta la partícula. Habrás experimentado esto como la sensación que sientes cuando un automóvil dobla una esquina o acelera.

In [ ]:
# Haremos un muestreo de las ecuaciones de movimiento en pasos de tiempo discretos.
 
dt         = 1.e-2
nsteps     = 500

timesteps  = np.linspace(0.0, nsteps * dt, nsteps)
dt         = np.diff(timesteps)[0]

mass       = 0.25

Ahora simplemente tenemos que ejecutar la simulación,

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,8))

ax.tick_params(labelbottom=False, labelleft=False)  

plt.axis('off')

# Reinitialise particles.
pos, vel = init_dof(npt=50)

for index_in_timestep , time in enumerate(timesteps):    
    pl.cla()
    
    ax.text(0.01, 1.05, '$t={:d}$'.format(index_in_timestep), transform=ax.transAxes)

    dvel                        = np.zeros_like(vel)
    dpos                        = np.zeros_like(pos)
    
    for index_in_particle, _ in enumerate(pos):
        g_X                     = g_at_particle(index_in_particle, pos, mass, doimages=True)
        
        # Update velocities. 
        dvel[index_in_particle] = dt * g_X 
        
        # Update positions.
        dpos[index_in_particle] = dt * vel[index_in_particle]           
        
    vel += dvel
    pos += dpos
    
    # Our simulation has periodic boundaries, if you go off one side you come back on the other!
    pos = pos % 1.
    
    ax.plot(pos[:,0], pos[:,1], '.', c='darkorange', alpha=0.75)
    
    ax.set_xlim(0., 1.)
    ax.set_ylim(0., 1.)
    
    fig.canvas.draw()

¡Intenta jugar con la configuración! Sin embargo, más de 100 partículas no funcionarán muy bien. Con la configuración predeterminada, encontrarás que las partículas tienden a caer en uno o dos cúmulos en poco tiempo. Esto se debe al arrastre que aplicamos. El arrastre simula el efecto que tiene el universo en expansión sobre las partículas reales de materia oscura, que es alentarlas y hacer que se agrupen. Estos grupos se conocen como halos y forman criaderos galácticos donde el gas puede acumularse para formar nuevas estrellas y galaxias.

Ahora, cuando DESI ejecuta grandes simulaciones, como las que se ejecutan en Summit, se resuelve un total de ~ 48 _billones de_ partículas. ¡No intentes esto aquí! Pero los resultados son realmente extraordinarios (¡salte al minuto 6 y 45 segundos si estás impaciente por ver el resultado!):

In [ ]:
YouTubeVideo('LQMLFryA_7k', width=800, height=400)

Con este gran éxito, viene una responsabilidad adicional. Esta informática de alto rendimiento, si bien es fantástica para DESI y la ciencia, ahora tiene una [huella de carbono](https://es.wikipedia.org/wiki/Huella_de_carbono) comparable a la [industria mundial de las aerolíneas](https://www.hpcwire.com/solution_content/ibm/cross-industry/five-tips-to-reduce-your-hpc-carbon-footprint/) ([aquí](https://porelclima.es/equipo/2873-centros-de-datos-sostenibles-reduciendo-la-huella-de-internet) un artículo similar en español) y consume la misma cantidad de electricidad que el país de Irán (¡82 millones de personas!).

Más preocupante aún, esto pronto crecerá del 2% del consumo de energía del mundo a ~ 30%. ¡Una tarifa extraordinaria!

Afortunadamente, Summit también se encuentra entre las supercomputadoras más ecológicas. Sus 14,7 GFlops/watt la pone número 1 en la [lista global Green 500 2019](https://www.top500.org/lists/green500/2019/06/). ¡Mejor suerte el año que viene, E.U.!

<img src="images/Amazon.jpg" alt="Drawing" style="width: 800px;"/>